# Get OSMnx Extraction

In [1]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import pickle

## Remote I/O

In [2]:
boundary_filename = "nw_pdx.geojson"
osm_filename = "Portland.osm"
pickle_filename = "osmnx-graph.pickle"

In [3]:
nw_pdx_gdf = gpd.read_file(boundary_filename)

## OSMnx Extraction

In [4]:
# Option 2: read from disk
graph = ox.graph_from_xml(osm_filename, simplify = False)

In [7]:
small_graph = ox.truncate.truncate_graph_bbox(
    graph, 
    nw_pdx_gdf.bounds.maxy[0], 
    nw_pdx_gdf.bounds.miny[0], 
    nw_pdx_gdf.bounds.maxx[0], 
    nw_pdx_gdf.bounds.minx[0]
)

In [11]:
link_gdf = ox.graph_to_gdfs(small_graph, nodes = False, edges = True)
link_gdf['osmid'] = (
 pd.DataFrame(link_gdf.osmid.tolist())
   .fillna('')
   .astype(str)
   .agg(','.join, 1)
   .str.strip(',')
)


In [12]:
link_gdf.head()

osmid                    name      highway maxspeed  \
u          v          key                                                       
1367343800 40541098   0     NaN    North Failing Street  residential   20 mph   
           7990710833 0     NaN    North Failing Street  residential   20 mph   
1367343801 7990710837 0     NaN    North Failing Street  residential   20 mph   
1367343802 40675211   0     NaN  North Vancouver Avenue    secondary   25 mph   
           7990710834 0     NaN                     NaN      footway      NaN   

                           oneway  length lanes service est_width width  \
u          v          key                                                 
1367343800 40541098   0     False  31.178   NaN     NaN       NaN   NaN   
           7990710833 0     False   0.804   NaN     NaN       NaN   NaN   
1367343801 7990710837 0     False   1.078   NaN     NaN       NaN   NaN   
1367343802 40675211   0      True   7.780     1     NaN       NaN   NaN   
           7990710834 0     False   5.240   NaN     NaN       NaN   NaN   

                          access landuse bridge  ref area tunnel junction  \
u          v          key                                                   
1367343800 40541098   0      NaN     NaN    NaN  NaN  NaN    NaN      NaN   
           7990710833 0      NaN     NaN    NaN  NaN  NaN    NaN      NaN   
1367343801 7990710837 0      NaN     NaN    NaN  NaN  NaN    NaN      NaN   
1367343802 40675211   0      NaN     NaN    NaN  NaN  NaN    NaN      NaN   
           7990710834 0      NaN     NaN    NaN  NaN  NaN    NaN      NaN   

                                                                    geometry  
u          v          key                                                     
1367343800 40541098   0    LINESTRING (-122.66808 45.55078, -122.66848 45...  
           7990710833 0    LINESTRING (-122.66808 45.55078, -122.66807 45...  
1367343801 7990710837 0    LINESTRING (-122.66784 45.55079, -122.66785 45...  
1367343802 40675211   0    LINESTRING (-122.66794 45.55086, -122.66794 45...  
           7990710834 0    LINESTRING (-122.66794 45.55086, -122.66787 45...

In [14]:
link_gdf = link_gdf.reset_index()

In [15]:
osmnx_gdf = link_gdf[['osmid', 'u', 'v', 'geometry']]
osmnx_gdf.to_file('osmnx_link_disk.geojson', driver='GeoJSON')